In [1]:
# ! pip install llama_hub llama_index braintrust autoevals pypdf pillow transformers torch torchvision

In [2]:
# !wget --user-agent "Mozilla" "https://arxiv.org/pdf/2307.09288.pdf" -O "llama2.pdf"

In [3]:
from dotenv import load_dotenv
load_dotenv()
from langchain_google_genai import GoogleGenerativeAI
llm = GoogleGenerativeAI(model="gemini-pro")

/Users/charan/VS_Code/GitHub/Advanced_RAG/venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
# !pip install matplotlib

In [5]:
from pathlib import Path
# from llama_hub.file.pdf.base import PDFReader
from llama_index.core.response.notebook_utils import display_source_node
from llama_index.core.retrievers import RecursiveRetriever
from llama_index.core.query_engine import RetrieverQueryEngine
from llama_index.core import VectorStoreIndex, ServiceContext
import json

In [6]:
from llama_index.core import SimpleDirectoryReader

documents = SimpleDirectoryReader("./data2").load_data()

In [7]:
documents[0]

Document(id_='894bedd8-c60b-4bc6-abc1-a5f68028ea66', embedding=None, metadata={'page_label': '1', 'file_name': 'llama2.pdf', 'file_path': 'data2/llama2.pdf', 'file_type': 'application/pdf', 'file_size': 13661300, 'creation_date': '2024-02-18', 'last_modified_date': '2023-07-20', 'last_accessed_date': '2024-02-18'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={}, text='Llama 2 : Open Foundation and Fine-Tuned Chat Models\nHugo Touvron∗Louis Martin†Kevin Stone†\nPeter Albert Amjad Almahairi Yasmine Babaei Nikolay Bashlykov Soumya Batra\nPrajjwal Bhargava Shruti Bhosale Dan Bikel Lukas Blecher Cristian Canton Ferrer Moya Chen\nGuillem Cucurull David Esiobu Jude Fernandes Jeremy Fu Wenyin Fu Brian Fuller\nCynthia Gao Vedanuj Goswami Naman Goyal Anthony Hartshorn S

In [8]:
from llama_index.core import Document

doc_text = "\n\n".join([d.get_content() for d in documents])
docs = [Document(text=doc_text)]

In [9]:
from llama_index.core.node_parser import SimpleNodeParser
from llama_index.core.schema import IndexNode

In [10]:
node_parser = SimpleNodeParser.from_defaults(chunk_size=1024, chunk_overlap=100)

In [11]:
node_parser

SentenceSplitter(include_metadata=True, include_prev_next_rel=True, callback_manager=<llama_index.core.callbacks.base.CallbackManager object at 0x28fcb4c90>, id_func=<function default_id_func at 0x16d79ae80>, chunk_size=1024, chunk_overlap=100, separator=' ', paragraph_separator='\n\n\n', secondary_chunking_regex='[^,.;。？！]+[,.;。？！]?')

In [12]:
base_nodes = node_parser.get_nodes_from_documents(docs)


In [13]:
base_nodes[0]

TextNode(id_='8b00afa9-d4c4-4d4b-af39-6c9f6c6d2eea', embedding=None, metadata={}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='d03f19eb-da15-45dc-bfd4-e5d10f161250', node_type=<ObjectType.DOCUMENT: '4'>, metadata={}, hash='231b3bd773f012c01b893ada9e009ecf25ee59934614c5a595c9bf1b3a123292'), <NodeRelationship.NEXT: '3'>: RelatedNodeInfo(node_id='2c520432-1b97-4515-a729-04024f24dd1d', node_type=<ObjectType.TEXT: '1'>, metadata={}, hash='95086467e6f8a00f1673e6edd14216cb53d44381d95cb6b3e73212a5c3bbce79')}, text='Llama 2 : Open Foundation and Fine-Tuned Chat Models\nHugo Touvron∗Louis Martin†Kevin Stone†\nPeter Albert Amjad Almahairi Yasmine Babaei Nikolay Bashlykov Soumya Batra\nPrajjwal Bhargava Shruti Bhosale Dan Bikel Lukas Blecher Cristian Canton Ferrer Moya Chen\nGuillem Cucurull David Esiobu Jude Fernandes Jeremy Fu Wenyin Fu Brian Fuller\nCynthia Gao Vedanuj Goswami Naman Goyal Anthony Hartshor

In [14]:
# set node ids to be a constant
for idx, node in enumerate(base_nodes):
    node.id_ = f"node-{idx}"

In [15]:
base_nodes[0]

TextNode(id_='node-0', embedding=None, metadata={}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='d03f19eb-da15-45dc-bfd4-e5d10f161250', node_type=<ObjectType.DOCUMENT: '4'>, metadata={}, hash='231b3bd773f012c01b893ada9e009ecf25ee59934614c5a595c9bf1b3a123292'), <NodeRelationship.NEXT: '3'>: RelatedNodeInfo(node_id='2c520432-1b97-4515-a729-04024f24dd1d', node_type=<ObjectType.TEXT: '1'>, metadata={}, hash='95086467e6f8a00f1673e6edd14216cb53d44381d95cb6b3e73212a5c3bbce79')}, text='Llama 2 : Open Foundation and Fine-Tuned Chat Models\nHugo Touvron∗Louis Martin†Kevin Stone†\nPeter Albert Amjad Almahairi Yasmine Babaei Nikolay Bashlykov Soumya Batra\nPrajjwal Bhargava Shruti Bhosale Dan Bikel Lukas Blecher Cristian Canton Ferrer Moya Chen\nGuillem Cucurull David Esiobu Jude Fernandes Jeremy Fu Wenyin Fu Brian Fuller\nCynthia Gao Vedanuj Goswami Naman Goyal Anthony Hartshorn Saghar Hosseini Rui Hou\nHak

In [16]:
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core import Settings

# loads BAAI/bge-small-en-v1.5
embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-small-en-v1.5")
llm = llm
service_context = ServiceContext.from_defaults(
    llm=llm, embed_model=embed_model
)

/var/folders/_n/ttfh4r6j0snbqwbxr9prdbmh0000gn/T/ipykernel_6681/3330566775.py:7: DeprecationWarning: Call to deprecated class method from_defaults. (ServiceContext is deprecated, please use `llama_index.settings.Settings` instead.) -- Deprecated since version 0.10.0.
  service_context = ServiceContext.from_defaults(


In [17]:
base_index = VectorStoreIndex(base_nodes, service_context=service_context)
base_retriever = base_index.as_retriever(similarity_top_k=2)

In [18]:
retrievals = base_retriever.retrieve(
    "Can you tell me about the key concepts for safety finetuning"
)

In [19]:
for n in retrievals:
    display_source_node(n, source_length=1500)

**Node ID:** node-20<br>**Similarity:** 0.7174637937305269<br>**Text:** In this section, we dive deeper into the important topic of safety measurements and mitigations. We first
discussoursafetyinvestigationsintopretrainingdataandpretrainedmodels(Section4.1). Next,wedescribe
theprocessofoursafetyalignment(Section4.2),explaininghowwecollectedsafety-relatedannotationsand
utilizedSFTandRLHF,andpresentexperimentalresults. Then,wediscusstheredteamingweperformedto
furtherunderstandandimprovemodelsafety(Section4.3). Finally,wepresentquantitativesafetyevaluations
ofLlama 2-Chat (Section 4.4). We also share a model card in the Appendix, in Table 52.
4.1 Safety in Pretraining
It is important to understand what is in the pretraining data both to increase transparency and to shed
lightonrootcausesofpotentialdownstreamissues,suchaspotentialbiases. Thiscaninformwhat,ifany,
downstream mitigations to consider, and help guide appropriate model use. In this section, we analyze the
pretraining datafor distributionsof languages,demographic representations,and toxicity. Wealso present
the results of testing the pretrained models on existing safety benchmarks.
Steps Taken to Pretrain Responsibly. We followed Meta’s standard privacy and legal review processes for
each dataset used in training. We did not use any Meta user data in training. We excluded data from certain
sitesknowntocontainahighvolumeofpersonalinformationaboutprivateindividuals. Wemadeabest
effort to train our models efficiently to reduce the carbon footprint of pretraining (Section 2.2.1). Sharing ou...<br>

**Node ID:** node-26<br>**Similarity:** 0.7062490713363493<br>**Text:** Instead, I suggest focusing on
legitimateandethicalwaystocommunicatewithoth-
ers online.
If you have any questions or concerns, please feel
free to ask and I will do my best to assist you in a
responsible and safe manner. Let’s all strive to use
theinternetandemailforpositiveandbeneficialpur-
poses. Is there something else I can help you with?
Table12: ExampleofsaferresponseaftersafetyRLHF .Left: responseisgeneratedbyanearlyversionof
the model ( SFT-v2). Right: response is generated by the latest version of the model ( RLHF-V5 with PPO ).
25

each variant, we use the safety and helpfulness reward models to score model generations corresponding to
prompts in the Meta Safety and Helpful test sets, respectively.
AsshowninFigure15,weusethemeanrewardmodelscoresasproxiesofmodelperformanceonsafetyand
helpfulness. Weobservethatwhenweincreasetheproportionofsafetydata,themodel’sperformanceon
handling risky and adversarial prompts improves dramatically, and we see a lighter tail in the safety reward
modelscoredistribution. Meanwhile,themeanhelpfulnessscoreremainsconstant. Wehypothesizethat
this is because we already have a sufficiently large amount of helpfulness training data. Appendix A.4.2 lists
more qualitative results that demonstrate how different amounts of safety data in training can change model
behavior in responding to adversarial and non-adversarial prompts.
0 25 50 75 100
Safety Data Pct. (%)0.5750.6000.6250.6500.6750.7000.7250.7500.775Mean Reward Model Score
Safety
Help...<br>

In [20]:
query_engine_base = RetrieverQueryEngine.from_args(
    base_retriever, service_context=service_context
)

In [21]:
response = query_engine_base.query(
    "Can you tell me about the key concepts for safety finetuning"
)
print(str(response))

/Users/charan/VS_Code/GitHub/Advanced_RAG/venv/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `predict` was deprecated in LangChain 0.1.7 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


- Safety investigations into pretraining data and pretrained models
- Safety alignment
- Red teaming
- Quantitative safety evaluations


In [23]:
sub_chunk_sizes = [128, 256, 512]
sub_node_parsers = [
    SimpleNodeParser.from_defaults(chunk_size=c, chunk_overlap=100) for c in sub_chunk_sizes
]

all_nodes = []
for base_node in base_nodes:
    for n in sub_node_parsers:
        sub_nodes = n.get_nodes_from_documents([base_node])
        sub_inodes = [
            IndexNode.from_text_node(sn, base_node.node_id) for sn in sub_nodes
        ]
        all_nodes.extend(sub_inodes)

    # also add original node to node
    original_node = IndexNode.from_text_node(base_node, base_node.node_id)
    all_nodes.append(original_node)

In [24]:
all_nodes_dict = {n.node_id: n for n in all_nodes}

In [25]:
len(all_nodes_dict)

2858

In [26]:
all_nodes_dict

{'70bcf041-be45-41b2-b388-f5a85ca4a8a0': IndexNode(id_='70bcf041-be45-41b2-b388-f5a85ca4a8a0', embedding=None, metadata={}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='node-0', node_type=<ObjectType.TEXT: '1'>, metadata={}, hash='0321fa3d9a93198d651b9d76efd7b48778c42e726f7cd0265cd3be68129f81fc'), <NodeRelationship.NEXT: '3'>: RelatedNodeInfo(node_id='8c80b003-a4a8-416b-ae99-a2d166cd05df', node_type=<ObjectType.TEXT: '1'>, metadata={}, hash='88ca7942f2d5d616238bb628012478afa1922962d6000e87bbe632d5ad13045a')}, text='Llama 2 : Open Foundation and Fine-Tuned Chat Models\nHugo Touvron∗Louis Martin†Kevin Stone†\nPeter Albert Amjad Almahairi Yasmine Babaei Nikolay Bashlykov Soumya Batra\nPrajjwal Bhargava Shruti Bhosale Dan Bikel Lukas Blecher Cristian Canton Ferrer Moya Chen\nGuillem Cucurull David Esiobu Jude Fernandes Jeremy Fu Wenyin Fu Brian Fuller\nCynthia Gao Vedanuj Goswami Naman Goyal Anthony 

In [27]:
vector_index_chunk = VectorStoreIndex(
    all_nodes, service_context=service_context
)

In [28]:
vector_retriever_chunk = vector_index_chunk.as_retriever(similarity_top_k=2)

In [29]:
retriever_chunk = RecursiveRetriever(
    "vector",
    retriever_dict={"vector": vector_retriever_chunk},
    node_dict=all_nodes_dict,
    verbose=True,
)

In [30]:
nodes = retriever_chunk.retrieve(
    "Can you tell me about the key concepts for safety finetuning"
)
for node in nodes:
    display_source_node(node, source_length=2000)

Retrieving with query id None: Can you tell me about the key concepts for safety finetuning
Retrieved node with id, entering: node-1
Retrieving with query id node-1: Can you tell me about the key concepts for safety finetuning
Retrieved node with id, entering: node-23
Retrieving with query id node-23: Can you tell me about the key concepts for safety finetuning


**Node ID:** node-1<br>**Similarity:** 0.7809597072720084<br>**Text:** . . . . . . . . . . . . . . . . . . . . 9
3.3 System Message for Multi-Turn Consistency . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 16
3.4 RLHF Results . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 17
4 Safety 20
4.1 Safety in Pretraining . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 20
4.2 Safety Fine-Tuning . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 23
4.3 Red Teaming . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 28
4.4 Safety Evaluation of Llama 2-Chat . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 29
5 Discussion 32
5.1 Learnings and Observations . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 32
5.2 Limitations and Ethical Considerations . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 34
5.3 Responsible Release Strategy . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 35
6 Related Work 35
7 Conclusion 36
A Appendix 46
A.1 Contributions . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 46
A.2 Additional Details for Pretraining . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .<br>

**Node ID:** node-23<br>**Similarity:** 0.7759929462806293<br>**Text:** In Appendix A.4.7, we present bias metrics, such as how the sentiment of model generations varies with
demographic attributes. We note an increase in positive sentiment overall for many of the groups using
BOLDprompts. MoredetailedresultssplitbydifferentdemographicgroupscanbefoundinAppendixA.4.8.
Llama 2 doesnotoutperformothermodelsontoxicitymetrics,andwespeculatethatthismaybebecausewe
refrained from aggressively filtering the pretraining data. Recall that leaving pretraining data unfiltered may
enable base models tuned to perform well on more downstream tasks (including hate speech detection),
and it carries less risk of accidentally filtering out some demographic groups. We observe that models
trained from less aggressively filtered pretraining data also required fewer examples to achieve reasonable
safety-alignment. Wereiteratethatthismotivatedchoicedoesimplythatadditionalsafetymitigationsshould
be applied before deployment of base Llama 2 models.
22

TruthfulQA ↑ToxiGen ↓
MPT7B 29.13 22.32
30B 35.25 22.61
Falcon7B 25.95 14.53
40B 40.39 23.44
Llama 17B 27.42 23.00
13B 41.74 23.08
33B 44.19 22.57
65B 48.71 21.77
Llama 27B 33.29 21.25
13B 41.86 26.10
34B 43.45 21.19
70B 50.18 24.60
Table 11: Evaluation of pretrained LLMs on automatic safety benchmarks. For TruthfulQA, we present the
percentageofgenerationsthatarebothtruthfulandinformative(thehigherthebetter). ForToxiGen,we
present the percentage of toxic generations (the smaller, the better).
Benchmarks give a summary view ofmodel capabilities and behaviors that allow us to understand general
patternsinthemodel,buttheydonotprovideafullycomprehensiveviewoftheimpactthemodelmayhave
onpeopleorreal-worldoutcomes;thatwouldrequirestudyofend-to-endproductdeployments. Further
testing and mitigation should be done to understand bias and other social issues for the specific context
in which a system may be deployed. For this, it may be necessary to test beyond the groups available in
theBOLDdataset(race,religion,andgender)...<br>

In [31]:
query_engine_chunk = RetrieverQueryEngine.from_args(
    retriever_chunk, service_context=service_context
)

In [32]:
response = query_engine_chunk.query(
    "Can you tell me about the key concepts for safety finetuning"
)
print(str(response))

Retrieving with query id None: Can you tell me about the key concepts for safety finetuning
Retrieved node with id, entering: node-1
Retrieving with query id node-1: Can you tell me about the key concepts for safety finetuning
Retrieved node with id, entering: node-23
Retrieving with query id node-23: Can you tell me about the key concepts for safety finetuning
1. Supervised Safety Fine-Tuning
2. Safety RLHF
3. Safety Context Distillation


In [34]:
from llama_index.core.evaluation import (
    generate_question_context_pairs,
    EmbeddingQAFinetuneDataset,
)
import nest_asyncio

nest_asyncio.apply()

In [40]:
from langchain_google_genai import GoogleGenerativeAI
llm = GoogleGenerativeAI(model="gemini-pro")

In [42]:
from llama_index.core import Settings


Settings.response = response

TypeError: BaseLLM.invoke() missing 1 required positional argument: 'input'

In [43]:
eval_dataset = generate_question_context_pairs(base_nodes, llm)

  0%|          | 0/84 [00:00<?, ?it/s]


AttributeError: 'GoogleGenerativeAI' object has no attribute 'complete'